In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandas_ml as pdml
import pyodbc

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier, IsolationForest, BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, roc_curve, roc_auc_score, average_precision_score, recall_score 
from sklearn.metrics import auc, precision_score, confusion_matrix, f1_score, precision_recall_curve
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import GridSearchCV

In [7]:
cnxn = pyodbc.connect(DSN='BGI40PROD')

cursor = cnxn.cursor()

In [3]:
sql = """select	CONSTITUENTSYSTEMID
        ,CONSTITUENTLOOKUPID
		,SORTNAMESHORT
		,CONSTITUENTAGE
from uif.v_DIM_CONSTITUENT
where ISINDIVIDUAL = 1 and ISDECEASED = 0 and ISACTIVE = 1"""

In [8]:
rfmsql = """select	rfm.CONSTITUENTSYSTEMID
		,rfm.DAYSDIFF
		,PERCENT_RANK () over (order by rfm.DAYSDIFF desc) as r_percentile
		,rfm.count_gifts
		,PERCENT_RANK () over (order by rfm.count_gifts asc) as f_percentile
		,rfm.total_giving
		,PERCENT_RANK () over (order by rfm.total_giving asc) as m_percentile
		,[RFM] = PERCENT_RANK () over (order by rfm.total_giving asc)  + PERCENT_RANK () over (order by rfm.count_gifts asc) + PERCENT_RANK () over (order by rfm.DAYSDIFF desc)
		
from

(select distinct a.CONSTITUENTSYSTEMID
		,convert(date,a.TargetDate) as TargetDate
		,DATEDIFF(dd,g.last_gift_date,a.TargetDate) as DAYSDIFF
		,isnull(g.count_gifts,0) as count_gifts
		,convert(date,g.last_gift_date) as RecentGiftDate
		,isnull(g.total_giving,0) as total_giving
from
(select CONSTITUENTSYSTEMID
		,convert(date,min(ORIGINALGIFTDATE)) as TargetDate
from [UIF].[v_DIM_PLANNEDGIFT]
where STATUS <> 'Withdrawn'
group by CONSTITUENTSYSTEMID

union

select CONSTITUENTSYSTEMID
		,convert(date,GETDATE()) as TargetDate
from uif.v_DIM_CONSTITUENT
where isindividual = 1 and isactive = 1) a

join (

select	donor_id
		,count_gifts
		,last_gift_date
		,total_giving

from
(
	select	a.[Recognized ConstituentSystemID] as donor_id
			,count(a.RevenueLookupID) as count_gifts
			,max(a.RevenueDate) as last_gift_date
			,sum(a.RecognitionAmount) as total_giving

		from [UIF].[GIVING_DETAIL_RECOGNITION] a
		join pm.v_DateAttributes da on a.RevenueDateDimID = da.DATEDIMID
		where	(a.RecognitionAmount > 0
				and a.REVENUERECOGNITIONTYPE in ('Primary','Shared','Prior Household Joint')
				and a.REVENUETRANSACTIONTYPE = 'Payment'
				and a.REVENUEAPPLICATION in ('Recurring Gift', 'Membership','Matching gift','Pledge')
				or (a.REVENUERECOGNITIONTYPE in ('Primary','Shared','Prior Household Joint')
					and a.RevenueTransactionType = 'Payment'
					and a.RevenueApplication = 'Donation'
					and a.CampusGiftType = 'Gift'
					and a.RecognitionAmount > 0))
				and exists (select a.*
							from (select distinct v.CONSTITUENTSYSTEMID as systemid
								from [VM].[v_Fact_DegreeArea] b
								join uif.v_DIM_CONSTITUENT v on b.ConstituentDimID = v.CONSTITUENTDIMID
								join [VM].[v_Dim_DegreeArea] c ON b.DegreeAreaDIMID = c.DegreeAreaDIMID
								where c.DegreeArea ='Law Grads'
					
								union

								select distinct q.ID as systemid
								from [dbo].[ADHOCQUERY_STATICIDSET_D9DB4F82_8EE5_4CC2_B35D_66A6BD369A38] q
								join uif.v_DIM_CONSTITUENT b on q.ID = b.CONSTITUENTSYSTEMID
								where b.ISINDIVIDUAL = 1 and b.ISACTIVE = 1) w where w.systemid = a.[Recognized ConstituentSystemID])
	group by a.[Recognized ConstituentSystemID]) a ) g on a.CONSTITUENTSYSTEMID = g.donor_id

	where exists (select a.*
							from (select distinct v.CONSTITUENTSYSTEMID as systemid
								from [VM].[v_Fact_DegreeArea] b
								join uif.v_DIM_CONSTITUENT v on b.ConstituentDimID = v.CONSTITUENTDIMID
								join [VM].[v_Dim_DegreeArea] c ON b.DegreeAreaDIMID = c.DegreeAreaDIMID
								where c.DegreeArea ='Law Grads'
					
								union

								select distinct q.ID as systemid
								from [dbo].[ADHOCQUERY_STATICIDSET_D9DB4F82_8EE5_4CC2_B35D_66A6BD369A38] q
								join uif.v_DIM_CONSTITUENT b on q.ID = b.CONSTITUENTSYSTEMID
								where b.ISINDIVIDUAL = 1 and b.ISACTIVE = 1) w where w.systemid = a.CONSTITUENTSYSTEMID)
			and (g.donor_id is null or g.last_gift_date < a.TargetDate) ) rfm"""

In [ ]:
app = pd.read_sql(sql, con=cnxn)

In [9]:
rfm = pd.read_sql(rfmsql, con=cnxn)

In [11]:
rfm = rfm.sort_values('RFM')

In [19]:
rfm['Rank_Pct']= rfm.RFM.rank(pct=True) * 100

In [24]:
rfm = rfm.sort_values('Rank_Pct', ascending=False)

In [26]:
rfm.to_csv("C:/Users/palmberg/Documents/GitHub/LawAFMAP/Data/rfm.csv")

In [ ]:
rfm.head()

In [ ]:
# pandas_ml confusion matrix: http://pandas-ml.readthedocs.io/en/latest/conf_mat.html

In [2]:
df = pd.read_csv("//uiffs01/dataanalytics/LawAffinityPG/lawdata_new1.csv")

df.fillna(0, inplace=True)  # we should do a better job of cleaning than this...

In [ ]:
list(df)

In [ ]:
#set a random seed variable
seed = 11251442

In [ ]:
# helper function to create column of bin values from big measures
# https://community.modeanalytics.com/python/tutorial/defining-python-functions/
def amount_bin (value):
    if value == 0 :
        return 'nonDonor'
    elif 0 < value < 100 :
        return 'donor_<100'
    elif 100 <= value < 250 :
        return 'donor_100-249'
    elif 250 <= value < 500 :
        return 'donor_250-499'
    elif 500 <= value < 1000 :
        return 'donor_500-999'
    elif 1000 <= value < 5000 :
        return 'donor_1000-4999'
    elif 5000 <= value < 10000 :
        return 'donor_5000-9999'
    elif 10000 <= value < 25000 :
        return 'donor_10000-24999'
    elif 25000 <= value < 50000 :
        return 'donor_25000-49999'
    return 'donor_>50000'

# helper function to create column of bin values from smaller measures
def count_bin (value):
    if value == 0 :
        return 'nonDonor'
    elif 0 < value < 9 :
        return '<10'
    elif 10 <= value < 20 :
        return '10-19'
    elif 20 <= value < 30 :
        return '20-29'
    elif 30 <= value < 40 :
        return '20-39'
    elif 40 <= value < 50 :
        return '40-49'
    elif 50 <= value < 60 :
        return '50-59'
    elif 60 <= value < 70 :
        return '60-69'
    elif 70 <= value < 80 :
        return '70-79'
    return '>80'

In [ ]:
#helper function to bin RFM rank into quintiles, lumping 0s into the bottom "quintile"
def rank_bin (value):
    if  0 <= value < 20 :
        return '20th'
    elif 20 <= value < 40 :
        return '40th'
    elif 40 <= value < 60 :
        return '60th'
    elif 60 <= value < 80 :
        return '80th'
    return '100th'

In [ ]:
#create new categorical fields using the helper functions
df['LWG_SUM_PRIOR_PG_BIN'] = df['LWG_SUM_BEFORE_PG'].apply(amount_bin)

df['NON_LWG_SUM_PRIOR_PG_BIN'] = df['NON_LWG_SUM_BEFORE_PG'].apply(amount_bin)

df['NON_LWG_GIFTYEARSCOUNT_PRIOR_PG_BIN'] = df['NON_LWG_GIFTYEARSCOUNT_PRIOR_PG'].apply(count_bin)

df['LWG_GIFTYEARSCOUNT_BEFORE_PG_BIN'] = df['LWG_GIFTYEARSCOUNT_BEFORE_PG'].apply(count_bin)

df['LWG_CONSECUTIVE_PRIOR_PG_BIN'] = df['LWG_CONSECUTIVE_PRIOR_PG'].apply(count_bin)

df['NON_LWG_CONSECUTIVE_PRIOR_PG_BIN'] = df['NON_LWG_CONSECUTIVE_PRIOR_PG'].apply(count_bin)

df['PRE_PG_YEARS_WITH_CONTACT_REPORT_BIN'] = df['PRE_PG_YEARS_WITH_CONTACT_REPORT'].apply(count_bin)

df['NON_LWG_CONSECUTIVE_PRIOR_PG_BIN'] = df['NON_LWG_CONSECUTIVE_PRIOR_PG'].apply(count_bin)

df['LWG_CONSECUTIVE_PRIOR_PG_BIN'] = df['LWG_CONSECUTIVE_PRIOR_PG'].apply(count_bin)

In [ ]:
#just checking a couple of the new features
df[['CONSTITUENTSYSTEMID','LWG_SUM_BEFORE_PG','LWG_SUM_PRIOR_PG_BIN', 'NON_LWG_SUM_BEFORE_PG', /
    'NON_LWG_SUM_PRIOR_PG_BIN']].head(50)

In [ ]:
#produce bar graph of one of the features created above to check distributions
df['LWG_GIFTYEARSCOUNT_BEFORE_PG_BIN'].value_counts().plot(kind='barh')

In [ ]:
#new column for Law giving as a % of total giving
df['LWG%'] = (df['LWG_SUM_BEFORE_PG'] / (df['LWG_SUM_BEFORE_PG'] + df['NON_LWG_SUM_BEFORE_PG'])) * 100

In [ ]:
#new column to indicate donors who've made more than 50% of their donations to Law
df['LWG>50%'] = (df['LWG_OVER_50%'] > 50.0).astype(int)

In [ ]:
#new column to indicate donors who've *only* given to Law
df['LWG_ONLY_DONOR'] = (df['LWG_OVER_50%'] == 100.0).astype(int)

In [ ]:
df['MLTPL_RESIDENCE_EVER'] = (df['RESIDENCE_COUNT_EVER'] > 1).astype(int)

In [ ]:
df['MLTPL_INVOLVEMENT'] = (df['INVOLVEMENT_COUNT'] > 1).astype(int)

df['MLTPL_INVOLVEMENT_TYPE'] = (df['INVOLVEMENT_TYPE_COUNT'] > 1).astype(int)

In [ ]:
#get dummies for all the categorical features I just made
pd.get_dummies(data=df, columns=['LWG_CONSECUTIVE_PRIOR_PG_BIN', 'NON_LWG_CONSECUTIVE_PRIOR_PG_BIN'])

In [ ]:
#drop the original measures used to create the categorical variables, as well as ID, dependent variable, and others
cols1 = df.drop(['CONSTITUENTSYSTEMID', 'HAS_PLANNED_GIFT', 'HAS_EMAIL', 'ISDECEASED', 'LWG_SUM_BEFORE_PG', 'NON_LWG_SUM_BEFORE_PG', 'PRE_PG_YEARS_WITH_CONTACT_REPORT', 'INVOLVEMENT_COUNT', 'INVOLVEMENT_TYPE_COUNT', 'RESIDENCE_COUNT_EVER', 'LWG_CONSECUTIVE_PRIOR_PG', 'NON_LWG_CONSECUTIVE_PRIOR_PG', 'LWG%'], axis = 1)

In [ ]:
#see which columns are of type 'object' so I can apply dummy encoding
list(cols1.select_dtypes(include='object'))

In [ ]:
#define X and y as dependent and independent variables
X = cols1
y = df.HAS_PLANNED_GIFT

X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, test_size=0.33, random_state=42)

[Tuning Random Forest Models](https://www.analyticsvidhya.com/blog/2015/06/tuning-random-forest-model/)

In [ ]:
#modified this to replace the random_state value with the pre-defined seed variable
print("X_train:", X_train.shape)

In [ ]:
clf = RandomForestClassifier(n_estimators=50, bootstrap=True, n_jobs=-1, random_state=seed, class_weight="balanced_subsample")
clf = clf.fit(X_train, y_train)

In [ ]:
selector = SelectFromModel(clf, threshold=".5*mean", prefit=True)
X_train2 = selector.transform(X_train)
X_test2 = selector.transform(X_test)
print("X_train2:", X_train2.shape)
print("X_test2:", X_test2.shape)

In [ ]:
def col_index(df):
    return {i: list(df.columns)[i] for i in range(len(df.columns))}

ind = col_index(X)

In [ ]:
n = X_train2.shape[1]

features = list(ind.values())
importances = clf.feature_importances_
indices = list(np.argsort(importances))[-n:]

plt.figure(figsize=(8,26))
plt.title('Feature Importances')
plt.barh(range(len(indices)), [importances[i] for i in indices], color='#FFCD00', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices]) 
plt.xlabel('Relative Importance');

In [ ]:
top = [ind[i] for i in indices[-n:]]

top.reverse()

top

In [ ]:
# modify the model params/values here for testing

rf = RandomForestClassifier(oob_score = True, n_jobs = -1,random_state =seed, max_features = "auto", class_weight="balanced_subsample")

param_grid = {"n_estimators": [1000, 1500, 2000],
              "max_depth": [None]
             }

gs = GridSearchCV(rf, param_grid=param_grid, scoring = 'average_precision', n_jobs=-1)

model = gs.fit(X_train2, y_train)

In [ ]:
model.cv_results_

In [ ]:
final_rf = model.best_estimator_

In [ ]:
final_rf = model.best_estimator_

final_rf.predict_proba(X_test2)

In [ ]:
fpr, tpr, thresholds = roc_curve(y_train, final_rf.predict_proba(X_train2)[:, 1])
roc_auc = auc(fpr, tpr)

Fpr, Tpr, Thresholds = roc_curve(y_test, final_rf.predict_proba(X_test2)[:, 1])
Roc_auc = auc(Fpr, Tpr)

plt.figure(figsize=(10,8))
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='Train ROC (area = %0.2f)' % roc_auc)
plt.plot(fpr, thresholds, color='darkorange', linestyle='-.')
plt.plot(Fpr, Tpr, color='gray',
         lw=lw, label='Test ROC (area = %0.2f)' % Roc_auc)
plt.plot(Fpr, Thresholds, color='gray', linestyle='-.', lw=3)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.savefig('roc.png')
plt.show();


In [ ]:
n = X_train2.shape[1]

features = list(ind.values())
importances = clf.feature_importances_
indices = list(np.argsort(importances))[-n:]

plt.figure(figsize=(8,26))
plt.title('Feature Importances')
plt.barh(range(len(indices)), [importances[i] for i in indices], color='#FFCD00', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices]) 
plt.xlabel('Relative Importance');

In [ ]:
plt.scatter(final_rf.predict_proba(X_test2)[:, 1], y_test)

In [ ]:
fpr, tpr, thresholds = roc_curve(y_train, final_rf.predict_proba(X_train2)[:, 1])
roc_auc = auc(fpr, tpr)

Fpr, Tpr, Thresholds = roc_curve(y_test, final_rf.predict_proba(X_test2)[:, 1])
Roc_auc = auc(Fpr, Tpr)

plt.figure(figsize=(10,8))
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='Train ROC (area = %0.2f)' % roc_auc)
plt.plot(fpr, thresholds, color='darkorange', linestyle='-.')
plt.plot(Fpr, Tpr, color='gray',
         lw=lw, label='Test ROC (area = %0.2f)' % Roc_auc)
plt.plot(Fpr, Thresholds, color='gray', linestyle='-.', lw=3)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.savefig('roc.png')
plt.show();


In [ ]:
nope = X[y == 0]

X2 = selector.transform(nope)

#col = ['ConstituentID', 'ConstituentAge', "SCORE"] +  cols

nope = nope.assign(SCORE = (final_rf.predict_proba(X2)[:,1]))

#nope.loc[:, col].sort_values(by="SCORE", ascending=False).head(50)

nope.sort_values(by="SCORE", ascending=False).head()

In [ ]:
#set series to_frame to enable merge

nope = pd.merge(nope, df.loc[df.ISDECEASED==0,"CONSTITUENTSYSTEMID"].to_frame(), how="inner", left_index=True, right_index=True )
nope.head(50)

In [ ]:
nope.sort_values(by="SCORE", ascending=False).head(100).to_clipboard()

In [ ]:
#shouldn't need this anymore, as transformed these features to categorical and then dummies

scaler = StandardScaler()

nope.loc[:, ['LWG_GIFTYEARSCOUNT_BEFORE_PG',
          'RESIDENCE_COUNT_EVER',
          'LWG_CONSECUTIVE_PRIOR_PG',
          'NON_LWG_CONSECUTIVE_PRIOR_PG',
          'NON_LWG_GIFTYEARSCOUNT_PRIOR_PG',
          'LWG_SUM_BEFORE_PG',
          'NON_LWG_SUM_BEFORE_PG',
          'PRE_PG_YEARS_WITH_CONTACT_REPORT',
          'INVOLVEMENT_COUNT',
          'INVOLVEMENT_TYPE_COUNT']] = \
scaler.inverse_transform(nope.loc[:, ['LWG_GIFTYEARSCOUNT_BEFORE_PG',
          'RESIDENCE_COUNT_EVER',
          'LWG_CONSECUTIVE_PRIOR_PG',
          'NON_LWG_CONSECUTIVE_PRIOR_PG',
          'NON_LWG_GIFTYEARSCOUNT_PRIOR_PG',
          'LWG_SUM_BEFORE_PG',
          'NON_LWG_SUM_BEFORE_PG',
          'PRE_PG_YEARS_WITH_CONTACT_REPORT',
          'INVOLVEMENT_COUNT',
          'INVOLVEMENT_TYPE_COUNT']])

In [ ]:
nope = pd.merge(nope, app, how='inner', on='CONSTITUENTSYSTEMID')

In [ ]:
nope.sort_values(by="SCORE", ascending=False).head(100).to_clipboard()

In [ ]:
clf.predict?

In [ ]:
# create objects to use to create a confusion matrix
# y_test = y_true y_pred = clf.predict_proba(X_test)

In [ ]:
pred = clf.predict_proba(X_test)

In [ ]:
# create a function to indicate whether column 1 in pred is > .5

In [ ]:
def threshold (predict):
    if predict >= .5 :
        return 1 
    else:
        return 0

In [ ]:
d = []

for i in range(pred.shape[0]):
    if pred[i, 1] >= .5:
        d.append((i, 0, pred[i, 1]))
    else:
        d.append((i,0,pred[i, 1]))
d[:10]       

In [ ]:
np.vectorize?

In [ ]:
np.apply_along_axis(threshold, 1, pred[:,1].reshape(-1,1))

In [ ]:
pred[:, 1].shape

In [ ]:
pred[:, 1].reshape

In [ ]:
pred[:, 1].reshape

In [ ]:
thresh = np.vectorize(threshold)

In [ ]:
thresh(pred[:, 1]).sum()

In [ ]:
confusion_matrix(y_test,thresh(pred[:, 1]))